<a href="https://colab.research.google.com/github/SatyamChoubey/Projects/blob/main/Graduate_Admission_prediction_with_pyspark_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TASK 1 : Install Dependencies & Run a SparkSession


In [6]:
#install pyspark
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=1f3e8c85aa0ec25dd153f9e6d4753726f1699e41846c97c53212ff44afe76871
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [8]:
#create a sparksession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('spark').getOrCreate()

# TASK 2 : Clone & Explore dataset

In [9]:
#clone the dataset
! git clone https://github.com/education454/admission_dataset

Cloning into 'admission_dataset'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), 5.60 KiB | 5.60 MiB/s, done.


In [10]:
#check the presence of dataset
! ls admission_dataset

Admission_Predict_Ver1.1.csv


In [14]:
#create a spark dataframe
df = spark.read.csv('/content/admission_dataset/Admission_Predict_Ver1.1.csv',header=True, inferSchema=True)

In [15]:
#display dataframe
df.show()

+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|Serial No|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|        1|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|        2|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|        3|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|        4|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|        5|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|        6|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|        7|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|        8|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|        9

In [17]:
#get the no.of rows & columns
print((df.count(),len(df.columns)))

(500, 9)


In [18]:
#print schema
df.printSchema()

root
 |-- Serial No: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [20]:
#get the summary statistics
df.describe().show()

+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|summary|        Serial No|         GRE Score|      TOEFL Score|University Rating|               SOP|               LOR|              CGPA|          Research|    Chance of Admit|
+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              500|               500|              500|              500|               500|               500|               500|               500|                500|
|   mean|            250.5|           316.472|          107.192|            3.114|             3.374|             3.484| 8.576440000000003|              0.56| 0.7217399999999996|
| stddev|144.4818327679989|11.295148372354712|6.081867659564538|1.143511800759815|0.9910036207566072|0.92

# TASK 3 : Data Cleaning

In [29]:
#drop the unnecessary column
df = df.drop('Serial No')

In [30]:
#display the dataframe
df.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|      302|        102|                1|2.0|1.5| 8.0|       0|            0.5|
|      323|        108|                3

In [31]:
#check for null values
for i in df.columns:
  print(i+":",df[df[i].isNull()].count())

GRE Score: 0
TOEFL Score: 0
University Rating: 0
SOP: 0
LOR: 0
CGPA: 0
Research: 0
Chance of Admit: 0


# TASK 4 : Correlation Analysis & Feature Selection

In [32]:
# correlation analysis
for col in df.columns:
  print('Correlation of chance of admit col for {} is {}'.format(col,df.stat.corr('Chance of Admit',col)))

Correlation of chance of admit col for GRE Score is 0.8103506354632598
Correlation of chance of admit col for TOEFL Score is 0.7922276143050823
Correlation of chance of admit col for University Rating is 0.6901323687886892
Correlation of chance of admit col for SOP is 0.6841365241316723
Correlation of chance of admit col for LOR is 0.6453645135280112
Correlation of chance of admit col for CGPA is 0.882412574904574
Correlation of chance of admit col for Research is 0.5458710294711379
Correlation of chance of admit col for Chance of Admit is 1.0


In [33]:
# feature selection
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['GRE Score', 'TOEFL Score',  'CGPA'], outputCol='features')

In [34]:
#display dataframe
output_data = assembler.transform(df)
output_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|          features|
+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|[337.0,118.0,9.65]|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|[324.0,107.0,8.87]|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72| [316.0,104.0,8.0]|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|[322.0,110.0,8.67]|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|[314.0,103.0,8.21]|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|[330.0,115.0,9.34]|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75| [321.0,109.0,8.2]|
|      308

# TASK 5 : Build the Linear Regression Model

In [35]:
#import Linearregression and create final data
from pyspark.ml.regression import LinearRegression
final_data = output_data.select('features','Chance of Admit')

In [36]:
#print schema of final data
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [38]:
#split the dataset into training and testing set
train,test = final_data.randomSplit([0.7,0.3])

In [39]:
#build & train the model
models = LinearRegression(featuresCol='features',labelCol='Chance of Admit')
model = models.fit(train)

In [41]:
#get coefficients & intercept
print("coefficients:",model.coefficients)
print("intercept:",model.intercept)

coefficients: [0.0025464887206850485,0.002782790264799685,0.14390311305533462]
intercept: -1.616840110750817


In [42]:
#get summary of the model
summary = model.summary

In [44]:
#print the rmse & r2 score
print("RMSE:",summary.rootMeanSquaredError)
print("R2 Score:",summary.r2)

RMSE: 0.06269348481239465
R2 Score: 0.7999429047455864


# TASK 6 : Evaluate & Save the Model

In [45]:
#transform on the test
predictions = model.transform(test)

In [46]:
#display the predictions
predictions.show(20)

+------------------+---------------+-------------------+
|          features|Chance of Admit|         prediction|
+------------------+---------------+-------------------+
|  [295.0,93.0,7.2]|           0.46|0.42927597047605226|
| [295.0,99.0,7.57]|           0.37|0.49921686389532405|
|  [296.0,97.0,7.8]|           0.49| 0.5292954880891367|
| [297.0,96.0,7.89]|           0.43| 0.5420104667200023|
| [298.0,97.0,7.21]|           0.45| 0.4494856288278595|
|  [298.0,99.0,7.6]|           0.46| 0.5111734234490395|
|[298.0,101.0,7.69]|           0.53| 0.5296902841536191|
|[298.0,101.0,7.86]|           0.54| 0.5541538133730259|
| [299.0,94.0,7.34]|           0.42|  0.462391151451339|
| [299.0,96.0,7.86]|           0.54| 0.5427863507697126|
|[299.0,100.0,7.88]|           0.51| 0.5567955740900177|
|[299.0,100.0,8.02]|           0.63| 0.5769420099177645|
|  [300.0,99.0,6.8]|           0.36| 0.4011439104461416|
| [300.0,99.0,8.01]|           0.58| 0.5752666772430965|
|[300.0,100.0,8.26]|           

In [47]:
#evaluate the model
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol='Chance of Admit',predictionCol='prediction',metricName='r2')

In [48]:
#save the model
model.save("model")

In [49]:
#load the model
from pyspark.ml.regression import LinearRegressionModel
loaded_model = LinearRegressionModel.load("model")